# **Titre du TP : Introduction à la Classification d'Images satellitaires avec les Réseaux Neuronaux Convolutifs (CNN)**
# CORRECTION de la partie Réalisation

**Description :**

Dans ce TP, nous explorerons les concepts fondamentaux liés à la classification d'images en utilisant des Réseaux Neuronaux Convolutifs (CNN). Notre objectif principal sera de catégoriser des images satellites en fonction du type de terrain.



**Étapes du TP :**

Dans ce TP nous allons apprendre à manipuler un réseau de neurones convolutionnel pour une tache de classification afin catégoriser des d'images satellites en fonction du type de terrain.

Nous allons progresser en suivant les étapes suivantes :

**Partie 1 :** Préparation de l'environnement de travail

1. **Bibliothéques Deep Leraning :** Utilisation de la libraire `fastai`
2. **Données images :** Utilisation d'images EuroSAT RGB

**Partie 2 :** Appropriation

1. **Utiliser un modèle Deep-Learning pré-entrainé** :  Apprendre (1) à charger un modèle de réseau de neurones existant et (2) à exécuter une tache de classification sur une images de notre choix téléchargée.

2. **Manipuler un dataset** : Apprendre (1) à charger un dataset d'images dans depuis le disque pour entrainer un modèle ; (2) Savoir comment explorer les données d'un dataset.

3. **Voir la structure d'un modèle** :  Apprendre (1) à créer lun modèle basé sur une architecture type (resnet50) et (2) à visualiser toutes les couches du réseau avec leurs détails.

4. **Entrainer un modèle**  Apprendre (1) à réaliser une phase d'apprentissage d'un modèle ; (2) comprendre comment sauvegarder un modèle avec ces paramètres d'apprentissage ; (3) apprendre à visualiser la précision de prédiction de notre modèle et enfin (4) voir comment examiner les résultats et les erreurs du modèle.

**Partie 3 :** Réalisation

1. **Classifier une photo satellitaire choisie** :
Trouver une photo satellitaire de la Réunion, telecharger là et utiliser notre modèle pour la classifier.

2. **Enrichir un modèle** :
Enrichir les classes du modèle pour prendre en considération d'autres types de terrain.

3. **Bonus avec pytorch** :
Pour les étudiants avancés, reproduire les élements de ce TP avec la bibliothèque `pytorch` qui necessite un peu plus d'entrer dans l'architecture du réseau de neurones.

---




## 3.1. **Classifier une image satellitaire choisie**

### **Exercice 1 :** Utiliser notre modèle avec vos propres images

Il vous est demandé ici :

1. de trouver quelques photos satellitaires  
sur lesquels nous pourrons appliquer notre modèle de classification.

2. d'écrire le code python permettant de choisir et télécharger une image à traiter. <BR>
Pour cela, utilisez l'utilitaire `widget` définie dans `fastai.vision.widgets` et qui permet le téléchargement d'une image avec l'instruction `widgets.FileUpload()`.

3. D'appliquer notre modèle à cette image

### Exercice 1. - CORRECTION

In [ ]:
from fastai.vision.widgets import *

# Création du bouton widget pour le téléchargement
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
from fastai.vision.all import PILImage

# l'image téléchargée est stockée dans la variable 'image'
image = PILImage.create(btn_upload.data[-1])
image

In [ ]:
pred, pred_idx, probs = model.predict(image)
f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'


## 3.2. **Enrichir un modèle : Enrichir les classes du modèle pour prendre en considération d'autres types de terrain.**

### **Exercice 2.** Enrichir ce modèle en affinant les classes à détecter **

Notre modèle a été pré-entrainé pour caractériser des images dans les classes :
- Nature
- Humain

Il vous est demandé de pouv oir maintenant considérer les classes suivantes :
  - Forest,
  - River,
  - AnnualCrop,
  - HerbaceousVegetation,
  - Residential,
  - PermanentCrop,
  - Industrial,
  - SeaLake,
  - Pasture,
  - Highway

  Pour cela :
  
  **a)** Définissez une nouvelle fonction d'extraction des labels de données applicable au dataset des images EuroSAT,  
  **b)** Testez en montrant que votre dataloader a bien étiqueté vos images de votre dataset.


### EXERCICE 2. - CORRECTION

In [ ]:
# import initiaux 
import fastai
from fastai.vision.all import untar_data, Path
from fastai.vision.all import vision_learner, resnet50, accuracy
from fastai.vision.all import plt, L
from fastai.vision.all import ClassificationInterpretation

path = untar_data('http://madm.dfki.de/files/sentinel/EuroSAT.zip')
Path.BASE_PATH = path

In [ ]:
# a) Nouvelle fonction d'extraction des labels de données.
# ======================================================================
def label_func(fname):
    # Liste des classes appartenant à "zones naturelles"
    nature_classes = ['Forest', 'River', 'AnnualCrop', 'HerbaceousVegetation', 
                      'PermanentCrop', 'SeaLake', 'Pasture', 'Residential', 
                      'Industrial', 'Highway']
    for nc in nature_classes:
      if (nc in fname.name):
        return nc
    return 'Erreur'

blocks = DataBlock(
  blocks = (ImageBlock, CategoryBlock),
  get_items = get_image_files,
  splitter = RandomSplitter(valid_pct=0.2, seed=0),
  get_y = label_func,   
  batch_tfms = aug_transforms(mult=2)
)

In [ ]:
# Variante optimisée de la question a)
# ======================================================================
#   On utilise `RegexLabeller` pour extraire les étiquettes à partir de noms de fichiers correspondant à un motif régulier (regex).
#   Dans notre cas, cette expression régulière capture une partie du nom du fichier qui se trouve entre le début de la chaîne
#   et le dernier souligné avant une séquence de chiffres suivie de ".jpg". Cette partie capturée est utilisée comme label pour les données.
#   Par exemple, si le nom du fichier est "nature_123.jpg", le label extrait serait "nature".

blocks = DataBlock(
  blocks = (ImageBlock, CategoryBlock),
  get_items = get_image_files,
  splitter = RandomSplitter(valid_pct=0.2, seed=0),
  get_y = using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),  # *** Nouvelle fonction d'extraction des labels de données
  batch_tfms = aug_transforms(mult=2)
)

In [ ]:
# Création du data loader
dataloader = blocks.dataloaders(path)

# b). Test
# ========
dataloader.show_batch(nrows=2, ncols=5)

### **Exercice 3.** Entrainement de votre modéle, test avec vos images et analyse des résultats

### Exercice 3. - CORRECTION

In [ ]:
# Entrainement du modèle
# ======================================================================

# creer le modele en lui demandant une architecture de type `resnet50`
model = vision_learner(dataloader, resnet50, metrics=accuracy)

# lancer l'apprentissage 3 fois sur le dataset 
model.fine_tune(3)

In [ ]:
# Visualisation des données suite à l'entrainement
# ======================================================================

accuracy_values = L(model.recorder.values).itemgot(2)
plt.plot(accuracy_values);

In [ ]:
# Examinun des résultats et les erreurs du modèle

interpretation = ClassificationInterpretation.from_learner(model)
interpretation.plot_confusion_matrix(figsize=(12,12), dpi=60)
interpretation.most_confused(min_val=5)

# *Partie 4 du TP : Bonus*

Pour les étudiants avancés, reproduire les élements de ce TP avec la bibliothèque PyTorch qui nécessite un peu plus de rentrer dans les détails de l'entrainement du réseau de neurones.

Pytorch permet d'avoir plus de flexibilité dans l'entraînement, et de nombreux choix de design ne sont pas fait par défaut et nous reviennent.

### Exercice "Bonus" - CORRECTION

In [ ]:
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import os
from pathlib import Path
from PIL import Image
import requests
from zipfile import ZipFile
from io import BytesIO
from tqdm import tqdm

# Download and unzip data
url = 'http://madm.dfki.de/files/sentinel/EuroSAT.zip'  # Define the URL of the dataset zip file
response = requests.get(url)  # Send an HTTP GET request to download the zip file
zip_file = ZipFile(BytesIO(response.content))  # Create a ZipFile object from the downloaded content
zip_file.extractall('EuroSAT')  # Extract all the contents of the zip file to the 'EuroSAT' directory
path = '/content/EuroSAT/'  # Set the path to the extracted dataset directory

# Custom Dataset Class
class EuroSATDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir  # Set the root directory where the dataset is located
        self.transform = transform or (lambda x: x)  # Define a data transformation (or use the identity transformation)
        self.images = list(Path(root_dir).rglob("*.jpg"))  # Create a list of image file paths in the dataset
        self.class_to_idx = {'Human': 0, 'Nature': 1}  # Define a mapping of class names to class indices
        self.class_to_idx_2 = {'Forest': 0, 'River': 1, 'AnnualCrop': 2, 'HerbaceousVegetation': 3, 'PermanentCrop': 4, 'SeaLake': 5, 'Pasture': 6, 'Highway': 7, 'Industrial': 8, 'Residential': 9}  # Define a more detailed class mapping

    def __len__(self):
        return len(self.images)  # Return the total number of images in the dataset

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])  # Get the file path of the image
        image = self.transform(Image.open(img_name))  # Apply the transformation to the image
        label = self.label_func(img_name)  # Get the label for the image
        return image, label  # Return the transformed image and its label

    def label_func(self, fname):
        nature_classes = ['Forest', 'River', 'AnnualCrop', 'HerbaceousVegetation', 'PermanentCrop', 'SeaLake', 'Pasture']
        label = 'Nature' if any(nc in fname for nc in nature_classes) else 'Human'  # Determine if the image belongs to the 'Nature' or 'Human' class based on its file path
        return self.class_to_idx[label]  # Return the class index

    def label_func_2(self, fname):
        label = fname.split("/")[-1].split("_")[0]  # Extract the class label from the image file path
        return self.class_to_idx_2[label]  # Return the class index

# Transformations (basic one only)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image pixel values
])

# Dataset and DataLoader
dataset = EuroSATDataset(path, transform=transform)  # Create an instance of the EuroSATDataset with the specified transformation
train_size = int(0.8 * len(dataset))  # Calculate the size of the training set (80% of the dataset)
valid_size = len(dataset) - train_size  # Calculate the size of the validation set (remaining 20% of the dataset)
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])  # Split the dataset into training and validation sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # Create a DataLoader for the training set
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)  # Create a DataLoader for the validation set

# Model
model = models.resnet50(pretrained=True)  # Load a pre-trained ResNet-50 model
num_ftrs = model.fc.in_features  # Get the number of input features to the final fully connected layer
model.fc = torch.nn.Linear(num_ftrs, 2)  # Replace the final fully connected layer with a new one for binary classification (2 classes: Nature and Human)

model.requires_grad_(False)  # Set all model parameters to not require gradients
model.fc.requires_grad_(True)  # Set the parameters of the final fully connected layer to require gradients

# Training setup
criterion = torch.nn.CrossEntropyLoss()  # Define the loss function (Cross-Entropy Loss)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Define the optimizer (Adam) and learning rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Check if a GPU is available and set the device accordingly

# Training loop
model.to(device)  # Move the model to the selected device (CPU or GPU)
for epoch in range(3):  # Loop over a fixed number of training epochs (3 in this case)
    model.train()  # Set the model to training mode
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{3}', unit='batch'):  # Iterate over batches of training data with a progress bar
        inputs = inputs.to(device)  # Move the input data to the selected device
        labels = labels.to(device)  # Move the labels to the selected device
        optimizer.zero_grad()  # Set the gradients of model parameters to zero
        outputs = model(inputs)  # Forward pass to get model predictions
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Backpropagate the gradients
        optimizer.step()  # Update model parameters based on gradients

# Save the model
torch.save(model.state_dict(), 'model.pth')  # Save the trained model's state_dict to a file


In [ ]:
# Validate model
model.eval()
valid_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        valid_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {correct/total}")

---

<center>FIN DU TP<BR>

<img src="http://lim.univ-reunion.fr/staff/courdier/media/home_media/CC_BY_4_0.jpeg" width="5%">

</center>

---